In [ ]:
# https://github.com/tomasonjo/blogs/blob/master/neo4jdocs/neo4j_support_bot.ipynb
# %pip install langchain==0.0.142 openai==0.27.4 beautifulsoup4==4.12.2 chromadb==0.3.21

In [1]:
import os
import dotenv

import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd

from langchain.document_loaders import (
    # GitLoader,
    # YoutubeLoader,
    # DataFrameLoader,
    UnstructuredURLLoader,
)
from langchain.text_splitter import CharacterTextSplitter

import tiktoken

# from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool

from crawl import crawl, clean, strip_content

# loads .env file with your OPENAI_API_KEY
dotenv.load_dotenv()

True

In [ ]:
# # Regex pattern to match a URL
# HTTP_URL_PATTERN = r"^http[s]*://.+"

# # Define root domain to crawl
# DOMAIN = "elevate-sg.com"
# FULL_URL = "https://elevate-sg.com/"

### Embedding documents for the first time

In [ ]:
# seen = crawl(FULL_URL, HTTP_URL_PATTERN)
# print(len(seen))

In [ ]:
# clean = clean(seen)
# print(len(clean))

In [2]:
# urls = list(clean)

urls = [
    "https://elevate-sg.com/",
    "https://elevate-sg.com/industry",
    "https://elevate-sg.com/solutions?solutions=Contractors",
    "https://elevate-sg.com/case-studies",
    "https://elevate-sg.com/about",
    "https://elevate-sg.com/careers",
]

# Without headers User-Agent, results in forbidden 403
loader = UnstructuredURLLoader(urls=urls, headers={"User-Agent": "Mozilla/5.0"})
data = loader.load()
print(len(data))

6


In [3]:
for d in data:
    new_content = strip_content(d.page_content)
    d.page_content = new_content

In [4]:
# Define text chunk strategy
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separator=" ")
# Split documents into chunks
data_split = splitter.split_documents(data)
print(len(data_split))

11


In [5]:
data_split

[Document(page_content='Revolutionising the maintenance industry At Elevate Tech, we specialise in innovative predictive maintenance solutions that utilise IoT sensors, real-time data, and advanced A.I/M.L models. Our platform delivers valuable insights that help organisations reduce downtime, improve equipment performance, and optimise maintenance practices. Our commitment to providing our clients with cutting-edge solutions is driven by a deep understanding of the importance of efficient maintenance practices in ensuring cost savings and increased efficiency. Join us in our mission to revolutionise the maintenance industry through the adoption of predictive maintenance solutions. We invite you to get in touch with us to learn more about how our platform can help you optimise your assets and improve your overall maintenance practices. Learn more Reduce breakdowns, longer equipment lifespan and enhanced safety Longer time between maintenance, thereby reducing your maintenance fees\u200

In [6]:
# # Define embedding model
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# store = Chroma.from_documents(
#     data_split, embeddings, collection_name="elt", persist_directory="db"
# )

# store.persist()
# store = None

Using embedded DuckDB with persistence: data will be stored in: db


### Chroma DB persisted

In [8]:
# Define embedding model
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
store = Chroma(
    collection_name="elt", persist_directory="db", embedding_function=embeddings
)
store._collection

Using embedded DuckDB with persistence: data will be stored in: db


Collection(name=elt)